# H2O Model

 * Wrap a H2O model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/helm/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * [H2O](https://www.h2o.ai/download/)

```bash
pip install seldon-core
pip install sklearn
```

## Train locally
 

In [8]:
!mkdir -p experiment

In [9]:
import h2o

h2o.init()
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

path = (
    "https://s3.amazonaws.com/h2o-public-test-data/smalldata/prostate/prostate.csv.zip"
)
h2o_df = h2o.import_file(path)
h2o_df["CAPSULE"] = h2o_df["CAPSULE"].asfactor()
model = H2OGeneralizedLinearEstimator(family="binomial")
model.train(y="CAPSULE", x=["AGE", "RACE", "PSA", "GLEASON"], training_frame=h2o_df)
modelfile = model.download_mojo(path="./experiment/", get_genmodel_jar=False)
print("Model saved to " + modelfile)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_232"; OpenJDK Runtime Environment (build 1.8.0_232-b09); OpenJDK 64-Bit Server VM (build 25.232-b09, mixed mode)
  Starting server from /home/agm/.virtualenvs/seldon-core/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpbv5dpvpz
  JVM stdout: /tmp/tmpbv5dpvpz/h2o_agm_started_from_python.out
  JVM stderr: /tmp/tmpbv5dpvpz/h2o_agm_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/London
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,"21 days, 15 hours and 52 minutes"
H2O cluster name:,H2O_from_python_agm_ususoe
H2O cluster total nodes:,1
H2O cluster free memory:,6.896 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
Model saved to /home/agm/Seldon/seldon-core/examples/models/h2o_mojo/experiment/GLM_model_python_1578393746111_1.zip


In [10]:
!mv experiment/*.zip src/main/resources/model.zip

Wrap model using s2i

In [11]:
!s2i build . seldonio/seldon-core-s2i-java-build:0.1 h2o-test:0.1 --runtime-image seldonio/seldon-core-s2i-java-runtime:0.1

---> Installing application source...
[INFO] Scanning for projects...
Downloading: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-starter-parent/1.5.1.RELEASE/spring-boot-starter-parent-1.5.1.RELEASE.pom
Downloaded: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-starter-parent/1.5.1.RELEASE/spring-boot-starter-parent-1.5.1.RELEASE.pom (8 KB at 28.4 KB/sec)
Downloading: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-dependencies/1.5.1.RELEASE/spring-boot-dependencies-1.5.1.RELEASE.pom
Downloaded: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-dependencies/1.5.1.RELEASE/spring-boot-dependencies-1.5.1.RELEASE.pom (88 KB at 2562.5 KB/sec)
Downloading: https://repo.maven.apache.org/maven2/kr/motd/maven/os-maven-plugin/1.4.1.Final/os-maven-plugin-1.4.1.Final.pom
Downloaded: https://repo.maven.apache.org/maven2/kr/motd/maven/os-maven-plugin/1.4.1.Final/os-maven-plugin-1.4.1.Final.pom (7

In [12]:
!docker run --name "h2o_predictor" -d --rm -p 5000:5000 h2o-test:0.1

ef967412b7e90d92de373b7625d24d06739611af8c2cf68a1b43e73e5d6c3f83


Send some random features that conform to the contract

In [6]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[70.599 '0' '1' '2' 3.24]]
RECEIVED RESPONSE:
data {
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.9687857517177408
        }
        values {
          number_value: 0.031214248282259202
        }
      }
    }
  }
}




In [13]:
!docker rm h2o_predictor --force

h2o_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [8]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


## Setup Seldon Core

Use the notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

## Build model image and run predictions

In [26]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-java-build:0.1 h2o-test:0.1 --runtime-image seldonio/seldon-core-s2i-java-runtime:0.1

---> Installing application source...
[INFO] Scanning for projects...
Downloading: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-starter-parent/1.5.1.RELEASE/spring-boot-starter-parent-1.5.1.RELEASE.pom
Downloaded: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-starter-parent/1.5.1.RELEASE/spring-boot-starter-parent-1.5.1.RELEASE.pom (8 KB at 20.6 KB/sec)
Downloading: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-dependencies/1.5.1.RELEASE/spring-boot-dependencies-1.5.1.RELEASE.pom
Downloaded: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-dependencies/1.5.1.RELEASE/spring-boot-dependencies-1.5.1.RELEASE.pom (88 KB at 936.8 KB/sec)
Downloading: https://repo.maven.apache.org/maven2/kr/motd/maven/os-maven-plugin/1.4.1.Final/os-maven-plugin-1.4.1.Final.pom
Downloaded: https://repo.maven.apache.org/maven2/kr/motd/maven/os-maven-plugin/1.4.1.Final/os-maven-plugin-1.4.1.Final.pom (7 

In [19]:
!kubectl create -f h2o_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


Wait until ready (replicas == replicasAvailable)

In [21]:
!kubectl rollout status deployment/h2o-deployment-h2o-predictor-1cc70ed

deployment "h2o-deployment-h2o-predictor-1cc70ed" successfully rolled out


In [25]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    seldon-deployment-example --namespace seldon -p

----------------------------------------
SENDING NEW REQUEST:

[[66.008 '1' '1' '0' 7.051]]
RECEIVED RESPONSE:
meta {
  puid: "d36r5qsei3dd62tkrt6g9i75h5"
  requestPath {
    key: "prostate-classifier"
    value: "h2o-test:0.1"
  }
}
data {
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.42621511584615
        }
        values {
          number_value: 0.57378488415385
        }
      }
    }
  }
}




In [ ]:
!minikube delete